# NYC OpenData: Cluster Identifier
This script takes a dictionary of data sets scraped from NYC OpenData and tries to identify clusters of similar ones.

Based on the 9/16/2021 version of the scraping tool, the scraped data has the following format:
* **name** (string): the name of the item
* **link** (string): the link to the page with more information about the item
* **category** (string): the category of the item (e.g., *Education*)
* **type** (string): the type of the item (e.g., *Dataset*)
* **description** (string): a description of the item
* **tags** (list of strings): a set of tags associated with the item
* **updated** (string): the timestamp which this item was last updated
* **apiDocLink** (string): a link to the API documentation (which might possibly be used to extract more metadata about the item)
* **columns**: a list of columns and their properties
    * **name** (string): the name of the column
    * **humanName** (string): a human-friendly name for the column
    * **type** (string): the column type (one of: `calendar_date`, `checkbox`, `location`, `number`, `point`, `text`, or `url`)
* **attachments**: a list of downloadable files—most often spreadsheets or PDFs—that describe the data in more detail
* **dataDownloads**: a list of downloadable files containing all of the data in different formats
* **rawMetadata**: a dictionary containing some of the raw data scraped from the page identified by *link*

## Installing and Loading Libraries

### Natural Language Toolkit (NLTK)

In [ ]:
!pip install nltk

import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

nltk.download('averaged_perceptron_tagger') # Needed for part of speech tagger
nltk.download('punkt') # Needed for lemmatizer
nltk.download('wordnet')

lemmatize_word = WordNetLemmatizer().lemmatize

### sci-kit learn

In [ ]:
from sklearn.cluster import AgglomerativeClustering

### The Usual

In [ ]:
import re
import json
import numpy as np
import pandas as pd
from functools import partial, reduce
from itertools import chain, combinations, permutations, starmap

### Extra Functions

In [ ]:
def apply_1(func, arg): # Applies 1 argument to a function
    return func(arg)

def compose(*funcs): # Composes one or more functions into one new function
    return reduce(lambda f, g: lambda x: f(g(x)), funcs)

def flat_map(func, iterable): # Flattens a sequence of sequences down one dimention and maps each of the elements
    return map(func, chain.from_iterable(iterable))

def map_apply(funcs, iterable): # Applies a sequence of functions to a sequence of sequences
    for args in iterable:
        yield tuple(starmap(apply_1, zip(funcs, args)))

def map_collect(map_func, collect_func, iterable): # Maps a sequence and applies the collect function to the result
    return collect_func(map(map_func, iterable))

def pairs(iterable): # Creates every 2-element permutation of a sequence
    return permutations(iterable, 2)

def first(iterable, default=None): # Gets the first element of a sequence or a default value
    return next(iter(iterable), default)

def second(iterable, default=None): # Gets the second element of a sequence or a default value
    iterator = iter(iterable)
    next(iterator, None)
    return next(iterator, default)

def zip_to_dict(keys, values): # Creates a new dictionary from a sequence of keys and a sequence of values
    return dict(zip(keys, values))

def load_from_cache(path): # Loads a JSON file from a given path
    with open(path, 'r') as cache_file:
        return json.load(cache_file)

## Loading Data

In [ ]:
#@title Settings
cache_path = "cache.json" #@param {type:"string"}

# Load the data sets.
data_sets = load_from_cache(cache_path)

# Display the names.
for data_set in data_sets.values():
    print('*', data_set['name'])

## Lemmatization

In [ ]:
# Helper functions for lemmatization
select_words = re.compile(r'yyyy(?:-yyyy)?|[0-9A-Za-z]+(?:\'s)?') \
                 .findall # re produces better results than nltk.tokenize.

substitute_years = compose( # Reduces years to a binary distinction, so different years have an edit distance of 0
    partial(re.compile(r'(?:^|\D)\d\d(?:\d\d)?(?:$|\D)').sub, ' yyyy '), # Just a year
    partial(re.compile(r'(?:^|\D)\d\d(?:\d\d)\s*-\s*\d\d(?:\d\d)?(?:$|\D)').sub, ' yyyy-yyyy ')) # A year range

def selectively_lower(value): # Lowercases strings that don’t look like acronyms
    if len(value) <= 3 or \
       value in ('yyyy', 'yyyy-yyyy') or \
       re.match(r'^.$|[A-Z][^a-z]|.[A-Z]$', value):
        return value
    return value.lower()

def convert_pos_tag_to_wordnet(tag): # Converts NLTK POS tags to WordNet POS tags
    tag = tag.upper()
    if tag in (',', ':', '(', ')', 'CC', 'CD', 'DT', 'IN', 'POS', 'TO'):
        return None # No POS—to ignore words that don’t contribute meaningfully to name comparison
    elif tag[0] == 'J':
        return wordnet.ADJ
    elif tag[0] == 'N':
        return wordnet.NOUN
    elif tag[0] == 'V':
        return wordnet.VERB
    elif tag[0] == 'R':
        return wordnet.ADV
    return wordnet.NOUN

def lemmatize_expression(expression): # Lemmatizes a string of words
    tokens = [*filter(None, # Filters out empty strings
                      select_words(substitute_years(expression)))]
    tagged_tokens = map_apply((selectively_lower,
                               convert_pos_tag_to_wordnet),
                              nltk.pos_tag(tokens))
    return [*starmap(lemmatize_word, # Feeds the converted token-POS pairs through the WordNet lemmatizer
                     filter(second, tagged_tokens))] # Filters out token-POS pairs with no POS

# Create a dictionary of data sets with their lemmatized names.
lemmatized_names = {identifier: lemmatize_expression(data_set['name'])
                    for identifier, data_set
                    in data_sets.items()}

# Display the unique lemmas found in the data set names.
unique_lemmas = {*flat_map(str.casefold,
                           lemmatized_names.values())}
print(f'{len(unique_lemmas):,} lemmas identified in {len(lemmatized_names):,} names:')
print(', '.join(sorted(unique_lemmas)))

## Edit Distances

In [ ]:
# Constants
INT_INF = 2147483647 # A stand-in for “very large number” that’s not a float; needed for the clustering algorithm

# First pass: compute the edit distance of all name pairs.
edit_distance_dict = zip_to_dict(pairs(lemmatized_names.keys()),
                                 starmap(nltk.edit_distance,
                                         pairs(lemmatized_names.values())))

# Second pass: set the edit distance of names with disjoint sets of lemmas to “infinity.”
# Names with few words will have short edit distances even though they have nothing in common, and
# resetting their distances to infinity lowers the ranking of such pairs, pushing them to the bottom.
for key1, key2 in edit_distance_dict:
    if set(lemmatized_names[key1]).isdisjoint(lemmatized_names[key2]):
        edit_distance_dict[(key1, key2)] = INT_INF

# Display the stats.
print(f'{len(edit_distance_dict):,} pairs of names measured')

## Clustering

In [ ]:
# Get the pairs with the minimum edit distance for any data set name.
def to_min_edit_distance_dict(edit_distance_dict):
    minimums = {} # Keeps track of the minimum edit distance for each name
    result = {}
    for (key1, key2), value in sorted(edit_distance_dict.items(), key=second):
        if value <= minimums.get(key1, INT_INF) and value <= minimums.get(key2, INT_INF):
            result[(key1, key2)] = value
            minimums[key1] = value
            minimums[key2] = value
    return result

min_edit_distance_dict = to_min_edit_distance_dict(edit_distance_dict)

# Display the edit distance stats.
# Note that each data set may have the same edit distance to more than one other data set,
# so the grand total of counts will be substantially more than the number of data sets.
edit_distance_counts = pd.Series(min_edit_distance_dict.values(),
                                 index=min_edit_distance_dict.keys()) \
                         .value_counts() \
                         .to_frame('Count') \
                         .sort_index() \
                         .sort_values(by='Count',
                                      ascending=False)
edit_distance_counts.index.set_names('Edit Distance', inplace=True)
edit_distance_counts.style.format('{:,}'.format)

In [ ]:
# Create a distance matrix from the edit distance dictionary.
D = pd.DataFrame(edit_distance_dict.values(),
                 index=edit_distance_dict.keys(),
                 columns=['Edit Distance']) \
      .unstack(1) \
      .droplevel(0, 1) \
      .fillna(0) # A name’s edit distance to itself will be 0.

# Cluster the data sets using the edit distances.
agg = AgglomerativeClustering(n_clusters=None,
                              distance_threshold=2, # Casts a wide net, but the clusters will be further refined.
                              affinity='precomputed',
                              linkage ='average')
clusters = pd.DataFrame(agg.fit_predict(D),
                        index=D.index,
                        columns=['cluster']) \
             .reset_index() \
             .groupby('cluster') \
             ['index'] \
             .agg(set) \
             .to_list()

# Sort the clusters by number of data sets within each of them.
clusters.sort(key=len, reverse=True)

# Display the cluster stats.
print(f'{len(clusters)} clusters in total')
print()
print('Top 3 clusters by data set count:')
for index in range(min(len(clusters), 3)):
    print(f'Cluster #{index}: {len(clusters[index]):,} data sets')
    print('*',
          map_collect(lambda identifier: data_sets[identifier]['name'],
                      '\n* '.join,
                      clusters[index]))